<a href="https://colab.research.google.com/github/bundickm/BoardGameGeek/blob/master/Final_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Container

In [0]:
import requests
import bs4
import pandas as pd
import time

In [0]:
from google.colab import files
upload = files.upload()

Saving BGG_IDs.csv to BGG_IDs.csv


In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [0]:
IDs = pd.read_csv('BGG_IDs.csv', header=None, squeeze=True)

In [0]:
def get_name(name_soup):
  for i in range(len(name_soup)):
    if (str(name_soup[i])[6] == 'p'):
      return name_soup[i].text

def maybe_get_soup(soup):
  if type(soup) == bs4.element.Tag:
    return soup.text
  else:
    return 'None'

def community_players(results, max_player):
  best, players = 0,0

  for i in range(len(results)//3):
    current = int(str(results[i*3])[18:-24])
    if (current > best):
      best = current
      players = i+1
  if (players > int(max_player)):
    players = str(players-1) + '+'
  return players

def language_dependance(results):
  best, dependance = 0,0
  
  for i in range(len(results)):
    votes = str(results[i])[28:38]
    while votes[0].isdigit() == False:
      votes = votes[1:]
    while votes.isdigit() == False:
      votes = votes[:len(votes)-1]
    if (int(votes) > best):
      dependance = i+1
      best = int(votes)
  return dependance

def community_age(results):
  best,age = 0,0
  for i in range(len(results)):
    votes = str(results[i])[18:28]
    while votes.isdigit() == False:
      votes = votes[:len(votes)-1]
    if (int(votes) > best):
      age = i
      best = int(votes)
  return age

def get_all(small_soup, ID):
  temp = pd.DataFrame(index=[ID], columns=[small_soup[i].text for i in range(len(small_soup))])
  temp[temp.index==ID] = True
  return temp

In [0]:
games_df = pd.DataFrame()
publishers,designers,artists = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
categories,mechanics,subdomains = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()

for ID in IDs:
  if ((ID != 235328) and (ID != 251214) and (ID != 99949) and (ID != 261838)):
    url = 'https://www.boardgamegeek.com/xmlapi/boardgame/'+str(ID)+'?stats=1'
    b_game = bs4.BeautifulSoup(requests.get(url).text).find('boardgame')

    polls = b_game.find_all('poll')
    rating = b_game.find('ratings')

    games_df = games_df.append({
      'id':(int(ID)),
      'name':(get_name(b_game.find_all('name'))),
      'publish_date':maybe_get_soup(b_game.find('yearpublished')),
      'honor_count':(len(b_game.find_all('boardgamehonor'))),
      'min_players':(b_game.find('minplayers').text),
      'max_players':(b_game.find('maxplayers').text),
      'age':(b_game.find('age').text),
      'avg_play_time':(b_game.find('playingtime').text),
      'min_play_time':(b_game.find('minplaytime').text),
      'max_play_time':(b_game.find('maxplaytime').text),
      'description':(b_game.find('description').text),
      'image':(str(b_game.find('image'))[7:-8]),
      'best_with':(community_players(polls[0].find_all('result'),
                                     (b_game.find('maxplayers').text))),
      'lang_dependance':(language_dependance(polls[1].find_all('result'))),
      'best_age':(community_age(polls[2].find_all('result'))),
      'rating_votes':(rating.find('usersrated').text),
      'rating':(rating.find('average').text),
      'bayes_rating':(rating.find('bayesaverage').text),
      'std_dev':(rating.find('stddev').text),
      'median':(rating.find('median').text),
      'owned':(rating.find('owned').text),
      'trading':(rating.find('trading').text),
      'wanting':(rating.find('wanting').text),
      'wishing':(rating.find('wishing').text),
      'comment_count':(rating.find('numcomments').text),
      'weight_votes':(rating.find('numweights').text),
      'weight':(rating.find('averageweight').text),
      }, ignore_index=True)

    publishers = publishers.append(get_all(
                  b_game.find_all('boardgamepublisher'),ID),sort=False)
    designers = designers.append(get_all(
                  b_game.find_all('boardgamedesigner'),int(ID)),sort=False)
    artists = artists.append(get_all(
                  b_game.find_all('boardgameartist'),int(ID)),sort=False)
    categories = categories.append(get_all(
                  b_game.find_all('boardgamecategory'),int(ID)),sort=False)
    mechanics = mechanics.append(get_all(
                  b_game.find_all('boardgamemechanic'),int(ID)),sort=False)
    subdomains = subdomains.append(get_all(
                  b_game.find_all('boardgamesubdomain'),int(ID)),sort=False)
    time.sleep(.25)

In [0]:
games_df.shape

In [0]:
mechanics.shape

In [0]:
games_df.to_csv('games_all.csv')
publishers.to_csv('publishers_all.csv')
designers.to_csv('designers_all.csv')
artists.to_csv('artists_all.csv')
categories.to_csv('categories_all.csv')
mechanics.to_csv('mechanics_all.csv')
subdomains.to_csv('subdomains_all.csv')